In [1]:
#Importamos las librerías que se van a utilizar 
from selenium import webdriver
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

In [3]:
#Establecermos algunas cosas antes
driver = webdriver.Chrome() #abrimos chrome
driver.maximize_window() # maximizamos la ventana emergente
url_1 = 'https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C' #establecemos el link de la página
driver.get(url_1) #hacemos que el link corra en la ventana emergente que abrimos


[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.78M/6.78M [00:13<00:00, 544kB/s]
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_17076/1874393190.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )


In [21]:
#Ahora establecemos algunas cosas
#---------------------------- Año
# Esteblecemos la dirección del botón para desplegar el menú de años
dropdown_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
dropdown_year.click()

# Ahora establecemos la lista de años disponibles 
list_year = driver.find_element( By.ID,  "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements( By.TAG_NAME, 'li') #si revisamos la página, la lista de años está etiquetada a partir de "li"
                                                        #entonces dentro de todo el menú de opciones en list_year, elegimos
                                                        #los que comparten el identificador "li"
list_year_options = len(years_list) #aquí optenemos la longitud de la lista de años antes determinada 

#------------------------Mes
#Hacemos lo mismo que hicimos para años, pero ahora para la opción de meses
dropdown_mes = driver.find_element( By.ID,  "ctl00_cphContent_rMes")
dropdown_mes.click()

# # List of months options
list_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements( By.TAG_NAME, 'li')
list_month_options = len(months_list)


In [16]:
year_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_year_options).tolist() )
#En la primera parte, en los arrays, se están concatenando dos valores (1 y 0)
#que representan las opciones todos y ninguno. Después, se agrega una secuencia de números 
# que representa los números de años, desde 2 hasta la longitud de la lista de opciones.
#En este caso, esa secuencia inicia desde 2, porque las opciones todos y ninguno
#ya están incluidas. Finalmente, la última parte convierte ese objeto array numpy a una lista,
#así, lo que tendrá el objeto "year_number_opt" es una lista que representa los números de años
#que se mostrarán en la lista de opciones.

month_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_month_options).tolist() ) #hacemos lo mismo para los meses

In [17]:
month_numbers_opt2 = np.array(list(range(1, 13)))
year_numbers_opt2 = np.array(list(range(1, 23)))

In [18]:
month_numbers_opt3 = np.array([1] + list(range(1, list_month_options)))

In [19]:
pwd

'C:\\Users\\Lenovo\\Documents\\GitHub\\New-try'

In [22]:
#Ahora, primero creamos listas vacías 
years_info_list = []
months_info_list = []
tables_info_list = []

for year_number in year_numbers_opt[0:22]: #aquí elegimos a los 22 años que existen en la lista de años (2002-2023) para la iteración
    
    # Dropdown button list
    dropdown_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio") #recodarmos el objeto ya definido para desplegar el menú de opciones de años
    dropdown_year.click() #hacemos click 

    time.sleep(3) #esperamos 3 segundos para iniciar el siguiente paso 

    
    driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click() #aquí encontramos 
    #la lista de años, es ese [] entraría 1,2,3,... según el años que elijamos
    #lo que se hace es que para cada iteración se aumente un año +1. Luego se indica que hagamos click
    year_text = driver.find_element( By.ID, "ctl00_cphContent_rAnio").text #obtenemos el texto dentro del elemento

    
        
    for month_number in month_numbers_opt[0:12]:#aquí elegimos los 12 meses que existen en la lista de meses para la iteración
        
        dropdown_mes = driver.find_element( By.ID, "ctl00_cphContent_rMes") #elegimos el boton de despliegue 
        dropdown_mes.click() #hacemos click
        
        time.sleep(2) #esperamos 2 segundos
        
        driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click() #lo mismo que hicimos 
        #para los años, pero ahora para los meses, incrementando el número para cada iteración
        month_text = driver.find_element( By.ID, "ctl00_cphContent_rMes").text #obtenemos el texto dentro del elemento
        print( year_text ) #para cada iteración se va a imprimir el año y mes a partir de ese texto antes establecido
        print( month_text )
        
        # En la página no es automática la consulta cuando eliges el año y mes, por eso hacemos click en el boton de consulta
        driver.find_element( By.ID,"ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)
        
        year_float = int(year_text) #aquí convertimos el objeto dentro de la cadena de texto en un número entero 
        
        try:
            # Aquí, primero se cambia el "frame" de la página, ese frame es una sección de la página que se puede cambiar
            #por separado del resto, en este caso es la tabla de contenido de la consulta. 
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            #luego se identifica a la tabla por el XPATH
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            #luego se obtiene el htlm de la tabla
            table_html = section_tab_table.get_attribute('outerHTML')
            #aquí se usa el pd.read_html para leer la tabla y convertirla en un objeto dataframe
            table_pd = pd.read_html( table_html )
            
             #aquí se limpia la tabla, se selecciona las filas a partir de la fila 5 y todas las columnas. Luego, se copia 
                #el resultado y se reinicia el índice, es decir, se ponde un nuevo índice simple de número secuenciales
            table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )
            
            
        #Finalmente, se cambia al "default_content" utilizando el método "driver.switch_to.default_content()"
        #y se imprime un mensaje de éxito o fracaso dependiendo de si se pudo encontrar la tabla o no. 
        #Si no se puede encontrar la tabla, se asigna la cadena "Null" a la variable "table_clean_1".
            driver.switch_to.default_content() #como antes había establecido el "enfocarnos" en la tabla, es decir a una parte del iframe de la página para
            #para poder editarla, ahora regresamos a toda la página 

            print("Table Tipo 1 - Cambiar de Frame y una tabla simple") #si lo anterior se cumple se va a imprimr este mensaje

        except:
            print("No table Tipo 1") #si no se encuentra una tabla así entonces se le asigna la cadena null a la tabla_clean_1
            table_clean_1 = "Null"
            
            
        try:
            # esto ya se explicó arriba
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )
 
            #aquí se estraen dos tablas del formato html antes establecido 
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_2 = table1.copy()

            # primero se obtiene el número de filas
            #de la segunda tabla, table1, y se genera un rango de valores para las filas que 
            #se quieren eliminar de la tabla, los valores desde 1 hasta el número de filas de table1 más 2
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # eliminamos esas filas antes dichas
            table0 = table0.drop( drop_rows, axis = 0 )

            # Aquí asignamos el valor del nombre de la columna para la tabla 
            # table_clean_2, así, primera sacamos el valor de la primera fila y primera columna 
            #de la tabla 0 (que contiene el nombre de la columna), le ponemos "col_name". 
            #Luego, extraemos el valor de la columna 0, comenzando desde la segunda fila de la tabla 0,
            #y se convierte en una lista. Por último, se inserta la lista como una nueva columna 
            #en la tabla table_clean_2 en la posición 0, con el nombre de la columna obtenido anteriormente.
            
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_2.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 2 - Cambiar de Frame y una tabla Compleja") #si esto sucede entonces imprimos este mensaje
            
             # Changing switch
            driver.switch_to.default_content() #lo explicamos antes
            
        except:
            print("No table Tipo 2") #lo explicamos antes
            table_clean_2 = "Null"
            

        try:
            # para esta parte es similar a la enterior, sol oque aquí no realizamos el cambio de frame
            section_id_table = driver.find_element( By.ID, "ctl00_cphContent_rpgActualMn_OT" )
            table_html = section_id_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            table0, table1 = table_pd[0], table_pd[1]
            table_clean_3 = table1.copy()


            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

  
            table0 = table0.drop( drop_rows, axis = 0 )
            

            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_3.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 3 - No Cambiar de Frame y una tabla Compleja")
            
        except:
            print("No table Tipo 3")
            table_clean_3 = "Null"
            
            
        # Check what to save
        if ( ( isinstance( table_clean_3, str ) == False ) and ( table_clean_3.shape[1] > 5 ) ):#aquí se comprueba que no sea un 
            #objeto str y que el tabla tenga más de 5 columnas
            
             # si eso se cumple, entonces se convierte a excel y se guarda en la dirección de trabajo, bajo el nombre del año y mes
                #de la itreación 
                        
            table_clean_3.to_excel( fr"C:\\Users\\Lenovo\\Documents\\GitHub\\New-try_{year_text}_{month_text}.xlsx", index = False , header = False )
            
            years_info_list.append( year_float) #aquí guardamos el año
            months_info_list.append( month_text ) #el mes
            tables_info_list.append( "Table 3" ) #y la tabla para tener un registro de la información extraída en cada iteración
            print("guardo3") #si todo ello se cumple imprimos que se guardo una tabla de tipo 3
            
        elif ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):
            
            # lo mismo de arriba, pero para el segundo tipo de tabla 
            table_clean_2.to_excel( fr"C:\\Users\\Lenovo\\Documents\\GitHub\\New-try_{year_text}_{month_text}.xlsx", index = False , header = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 2" )
            print("guardo2")
            
        elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
            # lo mismo para la tabla tipo 1
            table_clean_1.to_excel( fr"C:\\Users\\Lenovo\\Documents\\GitHub\\New-try_{year_text}_{month_text}.xlsx", index = False , header = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 1" )
            print("guardo1")
            
        # Si no hay información para ese año, como pasa para 2023 y parte de los años más antiguos, entonces
        else:
            # igual se lleva el registro del año y del mes sin registro y se guarda como "No table"
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "No Table" )
            
        driver.switch_to.default_content()

2022
Febrero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Enero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Marzo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Mayo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Julio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Agosto
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Setiembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2022
Octubre
No table Tipo 1
No

guardo3
2017
Octubre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2017
Noviembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2017
Diciembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Febrero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Enero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Marzo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Mayo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016
Julio
No tabl

2010
Mayo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Junio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Julio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Agosto
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Setiembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Octubre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Noviembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010
Diciembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009
Febrero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009
Enero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No t

2003
Febrero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Enero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Marzo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Abril
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Mayo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Junio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Julio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Agosto
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Setiembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003
Octubre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tip